In [2]:
import numpy as np
import numpy.linalg as linalg
import scipy as sp

In [3]:
# допоміжна функція для виводу матриці
def print_matrix(A, text):
    '''As argument takes matrix and text you want to ouput
    with matrix'''
    n,m = np.shape(A)
    print(text)
    for i in range(0,n):
        for j in range(0,m):
            print('{:>6}'.format(round(A[i,j], 3)), end = ' ')
        print()

In [4]:
# допоміжна функція для виводу розширеної матриці
def print_expanded_matrix(A,b, text):
    '''As argument takes matrix and text you want to ouput
    with matrix'''
    n,m = np.shape(A)  
    print(text)
    for i in range(0,n):
        for j in range(0,m):
            print('{:>6}'.format(round(A[i,j], 3)), end = ' ')
            if j == m-1:
                print('|{:>6}'.format(round(b[i])), end = ' ')
        print()

In [5]:
# допоміжна функція для виводу вектора
def print_vector(b, text):
    '''As argument takes matrix and text you want to ouput
    with matrix'''
    n=len(b)
    print(text)
    for i in range(0,n):
        print('{:>6}'.format(round(b[i], 3)))


# Метод Гауса

Метод Гауса розв'язання системи лінійних рівнянь проходить у два етапи: зведення системи до трикутного вигляду - "прямий хід" та зваротня підстановка - "обернений хід"

**Прямий хід ** виконується за формулами $$a_{ij}^{(k)} = a_{ij}^{(k-1)}  - \frac{a_{ik}^{(k-1)}}{a_{kk}^{(k-1)}} a_{kj}^{(k-1)}, b_i^{(k)} = b_i^{(k-1)}  - \frac{a_{ik}^{(k-1)}}{a_{kk}^{(k-1)}}  b_k^{(k-1)}  $$
 $k = 1,2,...n-1; j = k+1,...n$ $$a_{ij}^{(0)} = a_{ij}, b_i^{(0)} = b_i$$
 


In [6]:
def FeedForward(matrix,vector):
    n = np.shape(matrix)[0]
    A = np.copy(matrix)
    b = np.copy(vector)
    swapcount = 0
    errorcode = -1 #normal value of errorcode
    for i in range(0,n-1):
        if A[i,i] == 0:
            errorcode = i
            print("Спроба ділення на нуль у {} строці".format(i+1))
            return A,b,swapcount,errorcode
        maxindex = np.argmax(abs(A[i:,i])) #get index of maximum element in a column
        if(maxindex > i):
            A[[i,i+maxindex]]=A[[i+maxindex,i]] #swap rows if necessary
            b[[i,i+maxindex]]=b[[i+maxindex,i]]
            swapcount += 1
            print_expanded_matrix(A,b,"{} та {} рядки поміняні місцями".format(i+1,i+maxindex+1))
        for k in range(i+1,n):
            t = A[k,i]/A[i,i]
            b[k] =  b[k] - t*b[i]
            for j in range(i+1,n):
                A[k,j] = A[k,j] - t*A[i,j]
            A[k,i] = 0
        print_expanded_matrix(A,b,"Матриця А  після {} кроку".format(i+1))
    if A[n-1,n-1] == 0:
        errorcode = n-1
        return A,b,swapcount,errorcode
    else :
        return A,b,swapcount,errorcode
            

 **Обернений хід** виконується за формулами $$x_n = \frac{b_n}{a_{nn}} , x_i = \frac{1}{a_{ii}^{(i-1)}} \bigg( b_i^{(i-1)} - \sum_{j=i+1}^{n} a_{ij}^{(i-1)}x_j \bigg), j = n-1,n-2,...1 $$

In [7]:
def FeedBackward(matrix,vector):
    n = np.shape(matrix)[0]
    A = np.copy(matrix)
    b = np.copy(vector)
    x = np.zeros((n))
    x[n-1] = b[n-1]/A[n-1,n-1]
    for i in range(n-2,-1,-1):
        summ = 0
        for j in range(i+1,n):
            summ+= A[i,j]*x[j]
        x[i] = 1/A[i,i]*(b[i] - summ)
    return x
    

In [8]:
def Gaussian(A,b):
    print_expanded_matrix(A,b,"Початкова розширена матриця")
    matrix,vector,swapcount,errorcode = FeedForward(A,b)
    if errorcode == -1:
        return FeedBackward(matrix,vector)
    else:
        print("Спроба ділення на нуль у {} строці".format(errorcode+1))
        return errorcode

Над матрицею А було виконано лінійні перетворення, які не змінюють детермінанта матриці, отже він дорівнює детермінанту трикутної матриці, а саме $$detA =(-1)^p\displaystyle\prod_{i=1}^{n} a_{ii} $$, де р- кількість перестановок рядків

In [9]:
def DeterminantGaussian(A):
    n = np.shape(A)[0]
    A,b,swapcount,errorcode = FeedForward(A,np.zeros(n))
    det = 1
    if errorcode != -1:
        return 0
    for i in range(0,n):
        for j in range(0,n):
            if (i == j):
                det *= A[i,j]
    return ((-1)**swapcount) * det


**Обернена матриця** 
$$AX = E \Rightarrow X = A^{-1}E = A^{-1}$$
Представляємо $E = (e_1,e_2,...e_n)$, де $e_1 = (1; 0; 0;...;0)^T, e_2 = (0;1; 0;...;0)^T, ...,e_n = (0; 0; 0;...;1)^T$ і розв'язуємо n систем лінійних рівнянь $$AX=e_1,AX = e_2,...,AX = e_n$$ Тоді $$A^{-1} = (x_1,x_2,...,x_n)$$б де $x_i (i = 1,...n)$ - розв'язок відповідної СЛАР

In [10]:
def InvertedMatrixGaussian(A):
    n = np.shape(A)[0]
    I = np.eye(n)
    l = Gaussian(A,I[:,0])
    for i in range(1,n):
        l = np.vstack((l,Gaussian(A,I[:,i])))
    return l.T   

# Факторизація матриць
Факторизація або розклад матриці – її представлення у вигляді добутку декількох матриць.

**Теорема.** Якщо всі головні мінори матриці А відмінні від нуля, то існують такі нижня L і верхня U трикутні матриці, що A=LU. Якщо діагональні елементи однієї з матриць L або U фіксовані, то такий розклад єдиний.

$$u_{ij} = a_{ij} - \sum_{k=1}^{i-1} l_{ik}u_{kj}$$, де $i \leq j$; $u_{ij} = 0$ при $j<i$


$$l_{ij} = \frac{1}{u_{jj}} \bigg(a_{ij} -\sum_{k=1}^{j-1} l_{ik}u_{kj} \bigg)$$ , де $i > j$



In [11]:
def LUfactorization(A):
    n = np.shape(A)[0]
    L = np.eye(n)
    U = np.zeros((n,n))
    for i in range(0,n):
        for j in range(0,n):

            if i<= j :
                summ = 0
                for k in range(0,i):
                    summ += L[i,k]*U[k,j]
                U[i,j] = A[i,j] - summ
            elif i>j:
                U[i,j] = 0
                summ = 0
                for k in range (0,j):
                    summ += L[i,k]*U[k,j]
                L[i,j] = 1/U[j,j]*(A[i,j] - summ)
    return L,U

Після розкладання матриці А у добуток верхньої та нижньої трикутної матриці замість рівняння $Ax=b$ можна записати еквівалентне йому рівняння $LUx=b$.

Введемо вектор допоміжних змінних $y = (y_1,y_2,...,y_n)^T$ та перепишемо останнє рівняння у вигляді системи $$Ly=b,Ux=y$$
Після цього послідовно для $i = 1,2...,n$ знаходимо $$y_i = b_i - \sum_{k=1}^{i-1} l_{ik}y_k$$
У зворотньому порядку знаходимо невідомі за формулами $$ x_i = \frac{1}{u_{ii}}\bigg(y_i - \sum_{k=i+1}^{n} u_{ik}x_k \bigg), i = n,n-1,...1$$

In [12]:
def SolveLU(A,b):
    L,U = LUfactorization(A)
    n = len(b)
    y = np.zeros((n))
    x = np.zeros((n))
    for i in range(0,n):
        summ = 0 
        for k in range(0,i):
            summ += L[i,k]*y[k]
        y[i] = b[i] - summ
    for i in range(n-1,-1,-1):
        summ = 0
        for k in range(i+1,n):
            summ += U[i,k]*x[k]
        x[i] = 1/U[i,i]*(y[i] - summ)
    return x
            
            
    
    

Врахувавши властивості визначника і добутку матриць отримаємо $$detA = det(LU) = det(L)det(U) = \displaystyle\prod_{i=1}^{n} u_{ii} $$

$$A^{-1} = (LU)^{-1} = U^{-1}L^{-1}$$

In [13]:
def InvertedMatrixLU(A):
    L,U = LUfactorization(A)
    n = np.shape(A)[0]
    I = np.eye(n)
    Linv = FeedBackward(L.T,I[:,0])
    for i in range(1,n):
        Linv = np.vstack((Linv,FeedBackward(L.T,I[:,i])))  
    Uinv = FeedBackward(U,I[:,0])
    for i in range(1,n):
        Uinv = np.vstack((Uinv,FeedBackward(U,I[:,i]))) 
    return np.dot(Uinv.T,Linv)

In [14]:
def DeterminantLU(A):
    L,U = LUfactorization(A)
    n = np.shape(A)[0]
    det = 1
    for i in range(0,n):
        for j in range(0,n):
            if(i==j):
                det*=U[i,j]
    return det
                

# Основна програма


### Перша система

In [15]:
A1 = np.matrix([ [2.0,  0.0, 3.0,   1.0],
               [-4.0, 3.0, -4.0, -2.0],
               [4.0,  7.0, 9.0,  1.0],
               [5.0,  7.0, 0.0,  8.0]])
b1 = np.array([20.0,-34.0,48.0,97.0])

In [16]:
res = Gaussian(A1,b1)
print_vector(res, "Розв'язок системи")
cond1 = linalg.norm(A1)*linalg.norm(linalg.inv(A1))
print("cond={}".format(cond1))

Початкова розширена матриця
   2.0    0.0    3.0    1.0 |  20.0 
  -4.0    3.0   -4.0   -2.0 | -34.0 
   4.0    7.0    9.0    1.0 |  48.0 
   5.0    7.0    0.0    8.0 |  97.0 
1 та 4 рядки поміняні місцями
   5.0    7.0    0.0    8.0 |  97.0 
  -4.0    3.0   -4.0   -2.0 | -34.0 
   4.0    7.0    9.0    1.0 |  48.0 
   2.0    0.0    3.0    1.0 |  20.0 
Матриця А  після 1 кроку
   5.0    7.0    0.0    8.0 |  97.0 
   0.0    8.6   -4.0    4.4 |  44.0 
   0.0    1.4    9.0   -5.4 | -30.0 
   0.0   -2.8    3.0   -2.2 | -19.0 
Матриця А  після 2 кроку
   5.0    7.0    0.0    8.0 |  97.0 
   0.0    8.6   -4.0    4.4 |  44.0 
   0.0    0.0  9.651 -6.116 | -37.0 
   0.0    0.0  1.698 -0.767 |  -5.0 
Матриця А  після 3 кроку
   5.0    7.0    0.0    8.0 |  97.0 
   0.0    8.6   -4.0    4.4 |  44.0 
   0.0    0.0  9.651 -6.116 | -37.0 
   0.0    0.0    0.0  0.308 |   2.0 
Розв'язок системи
   7.0
   2.0
  -0.0
   6.0
cond=114.80255213436023


In [17]:
print_vector(linalg.solve(A1,b1), "Розв'язок системи, знайдений пакетною функцією")

Розв'язок системи, знайдений пакетною функцією
   7.0
   2.0
   0.0
   6.0


Знайдемо вектор нев'язок за формулою $r = Ax - b$

In [18]:
r = np.dot(A1,res) - b1
print_matrix(r.T,"Вектор нев'язок")


Вектор нев'язок
  -0.0 
   0.0 
   0.0 
   0.0 


    Отже, бачимо, що розв'язок точний, оскільки вектор нев'язки нульовий

Знайдемо LU розклад матриці A

In [19]:
L1,U1 = LUfactorization(A1)
print_matrix(L1,"Ліва нижня трикутна матриця")
print_matrix(U1,"Права верхня трикутна матриця")

Ліва нижня трикутна матриця
   1.0    0.0    0.0    0.0 
  -2.0    1.0    0.0    0.0 
   2.0  2.333    1.0    0.0 
   2.5  2.333    7.3    1.0 
Права верхня трикутна матриця
   2.0    0.0    3.0    1.0 
   0.0    3.0    2.0    0.0 
   0.0    0.0 -1.667   -1.0 
   0.0    0.0    0.0   12.8 


Перевіримо, чи дійсно знайдені матриці є LU розкладом початкової матриці А

In [20]:
print_matrix(L1.dot(U1),'Добуток матриць L та U')
print_matrix(A1, "Очікувана матриця А")

Добуток матриць L та U
   2.0    0.0    3.0    1.0 
  -4.0    3.0   -4.0   -2.0 
   4.0    7.0    9.0    1.0 
   5.0    7.0    0.0    8.0 
Очікувана матриця А
   2.0    0.0    3.0    1.0 
  -4.0    3.0   -4.0   -2.0 
   4.0    7.0    9.0    1.0 
   5.0    7.0    0.0    8.0 


Знайдемо розв'язок першої системи за допомогою  LU розкладу

In [21]:
print_vector(SolveLU(A1,b1), "Розв'язок, знайдений методом LU розкладу")

Розв'язок, знайдений методом LU розкладу
   7.0
   2.0
  -0.0
   6.0


Знайдемо детермінант матриці методом Гауса та LU

In [22]:
print("detA = {}".format(DeterminantGaussian(A1)))

1 та 4 рядки поміняні місцями
   5.0    7.0    0.0    8.0 |   0.0 
  -4.0    3.0   -4.0   -2.0 |   0.0 
   4.0    7.0    9.0    1.0 |   0.0 
   2.0    0.0    3.0    1.0 |   0.0 
Матриця А  після 1 кроку
   5.0    7.0    0.0    8.0 |   0.0 
   0.0    8.6   -4.0    4.4 |   0.0 
   0.0    1.4    9.0   -5.4 |   0.0 
   0.0   -2.8    3.0   -2.2 |   0.0 
Матриця А  після 2 кроку
   5.0    7.0    0.0    8.0 |   0.0 
   0.0    8.6   -4.0    4.4 |   0.0 
   0.0    0.0  9.651 -6.116 |   0.0 
   0.0    0.0  1.698 -0.767 |   0.0 
Матриця А  після 3 кроку
   5.0    7.0    0.0    8.0 |   0.0 
   0.0    8.6   -4.0    4.4 |   0.0 
   0.0    0.0  9.651 -6.116 |   0.0 
   0.0    0.0    0.0  0.308 |   0.0 
detA = -128.0


In [23]:
print("detA = {}".format(linalg.det(A1)))

detA = -127.99999999999997


In [24]:
print("detA = {}".format(DeterminantLU(A1)))

detA = -127.99999999999999


Знайдемо обернені матриці 

In [25]:
print_matrix(InvertedMatrixLU(A1),"Обернена матриця, знайдена за допомогою LU розкладу")

Обернена матриця, знайдена за допомогою LU розкладу
-4.203 -1.641  0.672  0.031 
-0.703 -0.141  0.172  0.031 
 2.055  0.711 -0.258 -0.047 
 3.242  1.148  -0.57  0.078 


In [26]:
print_matrix(linalg.inv(A1),"Обернена матриця, знайдена пакетом ")

Обернена матриця, знайдена пакетом 
-4.203 -1.641  0.672  0.031 
-0.703 -0.141  0.172  0.031 
 2.055  0.711 -0.258 -0.047 
 3.242  1.148  -0.57  0.078 


In [27]:
print_matrix(InvertedMatrixGaussian(A1),"Обернена матриця методом Гауса")

Початкова розширена матриця
   2.0    0.0    3.0    1.0 |   1.0 
  -4.0    3.0   -4.0   -2.0 |   0.0 
   4.0    7.0    9.0    1.0 |   0.0 
   5.0    7.0    0.0    8.0 |   0.0 
1 та 4 рядки поміняні місцями
   5.0    7.0    0.0    8.0 |   0.0 
  -4.0    3.0   -4.0   -2.0 |   0.0 
   4.0    7.0    9.0    1.0 |   0.0 
   2.0    0.0    3.0    1.0 |   1.0 
Матриця А  після 1 кроку
   5.0    7.0    0.0    8.0 |   0.0 
   0.0    8.6   -4.0    4.4 |   0.0 
   0.0    1.4    9.0   -5.4 |   0.0 
   0.0   -2.8    3.0   -2.2 |   1.0 
Матриця А  після 2 кроку
   5.0    7.0    0.0    8.0 |   0.0 
   0.0    8.6   -4.0    4.4 |   0.0 
   0.0    0.0  9.651 -6.116 |   0.0 
   0.0    0.0  1.698 -0.767 |   1.0 
Матриця А  після 3 кроку
   5.0    7.0    0.0    8.0 |   0.0 
   0.0    8.6   -4.0    4.4 |   0.0 
   0.0    0.0  9.651 -6.116 |   0.0 
   0.0    0.0    0.0  0.308 |   1.0 
Початкова розширена матриця
   2.0    0.0    3.0    1.0 |   0.0 
  -4.0    3.0   -4.0   -2.0 |   1.0 
   4.0    7.0    9.0    1

### Друга система

In [28]:
A2 = np.matrix([[-5.,0.,7.,0.],
                [4.,-24.,0.,1.],
                [3.,12.,-7.,-23.],
                [-2.,42.,37.,-21.]])
b2 = np.array([109., 168., -193., -95.])


In [29]:
res2 = Gaussian(A2,b2)
print_vector(res2, "Розв'язок системи")
cond2 = linalg.norm(A2)*linalg.norm(linalg.inv(A2))
print("cond={}".format(cond2))

Початкова розширена матриця
  -5.0    0.0    7.0    0.0 | 109.0 
   4.0  -24.0    0.0    1.0 | 168.0 
   3.0   12.0   -7.0  -23.0 |-193.0 
  -2.0   42.0   37.0  -21.0 | -95.0 
Матриця А  після 1 кроку
  -5.0    0.0    7.0    0.0 | 109.0 
   0.0  -24.0    5.6    1.0 | 255.0 
   0.0   12.0   -2.8  -23.0 |-128.0 
   0.0   42.0   34.2  -21.0 |-139.0 
2 та 4 рядки поміняні місцями
  -5.0    0.0    7.0    0.0 | 109.0 
   0.0   42.0   34.2  -21.0 |-139.0 
   0.0   12.0   -2.8  -23.0 |-128.0 
   0.0  -24.0    5.6    1.0 | 255.0 
Матриця А  після 2 кроку
  -5.0    0.0    7.0    0.0 | 109.0 
   0.0   42.0   34.2  -21.0 |-139.0 
   0.0    0.0 -12.571  -17.0 | -88.0 
   0.0    0.0 25.143  -11.0 | 176.0 
Матриця А  після 3 кроку
  -5.0    0.0    7.0    0.0 | 109.0 
   0.0   42.0   34.2  -21.0 |-139.0 
   0.0    0.0 -12.571  -17.0 | -88.0 
   0.0    0.0    0.0  -45.0 |  -0.0 
Розв'язок системи
 -12.0
  -9.0
   7.0
   0.0
cond=15.848709165610314


In [30]:
print_vector(linalg.solve(A2,b2), "Розв'язок системи, знайдений пакетною функцією")

Розв'язок системи, знайдений пакетною функцією
 -12.0
  -9.0
   7.0
  -0.0


Знайдемо вектор нев'язок за формулою $r = Ax - b$

In [31]:
r2 = np.dot(A2,res2) - b2
print_matrix(r2.T,"Вектор нев'язок")


Вектор нев'язок
   0.0 
  -0.0 
   0.0 
   0.0 


    Отже, бачимо, що розв'язок точний, оскільки вектор нев'язки нульовий

Знайдемо LU розклад матриці A

In [32]:
L2,U2 = LUfactorization(A2)
print_matrix(L2,"Ліва нижня трикутна матриця")
print_matrix(U2,"Права верхня трикутна матриця")

Ліва нижня трикутна матриця
   1.0    0.0    0.0    0.0 
  -0.8    1.0    0.0    0.0 
  -0.6   -0.5    1.0    0.0 
   0.4  -1.75 2.476979795053773e+16    1.0 
Права верхня трикутна матриця
  -5.0    0.0    7.0    0.0 
   0.0  -24.0    5.6    1.0 
   0.0    0.0    0.0  -22.5 
   0.0    0.0    0.0 5.573204538870989e+17 


Бачимо дуже великі значення. Перевіримо, чи дійсно знайдені матриці є LU розкладом початкової матриці А

In [33]:
print_matrix(L2.dot(U2),'Добуток матриць L та U')
print_matrix(A2, "Очікувана матриця А")

Добуток матриць L та U
  -5.0    0.0    7.0    0.0 
   4.0  -24.0    0.0    1.0 
   3.0   12.0   -7.0  -23.0 
  -2.0   42.0   37.0    0.0 
Очікувана матриця А
  -5.0    0.0    7.0    0.0 
   4.0  -24.0    0.0    1.0 
   3.0   12.0   -7.0  -23.0 
  -2.0   42.0   37.0  -21.0 


Отже, отримані значення  L та U не є розкладом початкової матриці. Це пояснюється тим, що при обраховуванні суми на 1 кроці  алгоритму (i = 1, j = 2) виникає множення 0.8 на 7 і в комп'ютері  це значення зберігається як 5.600000000005, а не 5.6. Якщо ввести поріг,з яким порівнювати значення U, в подальшому виникне 0, а потім - дялення на 0. 
 LU розкладу для цієї системи не існує, оскільки не виконується вимога, щоб усі кутові мінори не дорівнювали нулю,  а саме третій мінор = 0. 
 

Знайдемо детермінант матриці методом Гауса

In [34]:
print("detA = {}".format(DeterminantGaussian(A2)))

Матриця А  після 1 кроку
  -5.0    0.0    7.0    0.0 |   0.0 
   0.0  -24.0    5.6    1.0 |   0.0 
   0.0   12.0   -2.8  -23.0 |   0.0 
   0.0   42.0   34.2  -21.0 |   0.0 
2 та 4 рядки поміняні місцями
  -5.0    0.0    7.0    0.0 |   0.0 
   0.0   42.0   34.2  -21.0 |   0.0 
   0.0   12.0   -2.8  -23.0 |   0.0 
   0.0  -24.0    5.6    1.0 |   0.0 
Матриця А  після 2 кроку
  -5.0    0.0    7.0    0.0 |   0.0 
   0.0   42.0   34.2  -21.0 |   0.0 
   0.0    0.0 -12.571  -17.0 |   0.0 
   0.0    0.0 25.143  -11.0 |   0.0 
Матриця А  після 3 кроку
  -5.0    0.0    7.0    0.0 |   0.0 
   0.0   42.0   34.2  -21.0 |   0.0 
   0.0    0.0 -12.571  -17.0 |   0.0 
   0.0    0.0    0.0  -45.0 |   0.0 
detA = 118800.00000000001


In [35]:
print("detA = {}".format(linalg.det(A2)))

detA = 118799.99999999988


Знайдемо обернену матрицю

In [36]:
print_matrix(InvertedMatrixGaussian(A2),"Обернена матриця методом Гауса")

Початкова розширена матриця
  -5.0    0.0    7.0    0.0 |   1.0 
   4.0  -24.0    0.0    1.0 |   0.0 
   3.0   12.0   -7.0  -23.0 |   0.0 
  -2.0   42.0   37.0  -21.0 |   0.0 
Матриця А  після 1 кроку
  -5.0    0.0    7.0    0.0 |   1.0 
   0.0  -24.0    5.6    1.0 |   1.0 
   0.0   12.0   -2.8  -23.0 |   1.0 
   0.0   42.0   34.2  -21.0 |  -0.0 
2 та 4 рядки поміняні місцями
  -5.0    0.0    7.0    0.0 |   1.0 
   0.0   42.0   34.2  -21.0 |  -0.0 
   0.0   12.0   -2.8  -23.0 |   1.0 
   0.0  -24.0    5.6    1.0 |   1.0 
Матриця А  після 2 кроку
  -5.0    0.0    7.0    0.0 |   1.0 
   0.0   42.0   34.2  -21.0 |  -0.0 
   0.0    0.0 -12.571  -17.0 |   1.0 
   0.0    0.0 25.143  -11.0 |   1.0 
Матриця А  після 3 кроку
  -5.0    0.0    7.0    0.0 |   1.0 
   0.0   42.0   34.2  -21.0 |  -0.0 
   0.0    0.0 -12.571  -17.0 |   1.0 
   0.0    0.0    0.0  -45.0 |   2.0 
Початкова розширена матриця
  -5.0    0.0    7.0    0.0 |   0.0 
   4.0  -24.0    0.0    1.0 |   1.0 
   3.0   12.0   -7.0  -

In [37]:
print_matrix(linalg.inv(A2),"Обернена матриця, знайдена пакетом ")

Обернена матриця, знайдена пакетом 
-0.195  0.042 -0.027  0.032 
-0.034 -0.036 -0.006  0.005 
 0.003   0.03 -0.019  0.023 
-0.044 -0.022 -0.044   -0.0 


### Третя система

In [38]:
A3 = np.matrix([[5.,0.,-7.,0.],
                [-1.,6.,0.,1.],
                [2.,-6.,-4.,-5.],
                [-6.,-6.,15.,7.]])
b3 = np.array([-123.,60.,-108.,159.])

In [39]:
res3 = Gaussian(A3,b3)
print("cond={}".format(linalg.norm(A3)*linalg.norm(linalg.inv(A3))))

Початкова розширена матриця
   5.0    0.0   -7.0    0.0 |-123.0 
  -1.0    6.0    0.0    1.0 |  60.0 
   2.0   -6.0   -4.0   -5.0 |-108.0 
  -6.0   -6.0   15.0    7.0 | 159.0 
1 та 4 рядки поміняні місцями
  -6.0   -6.0   15.0    7.0 | 159.0 
  -1.0    6.0    0.0    1.0 |  60.0 
   2.0   -6.0   -4.0   -5.0 |-108.0 
   5.0    0.0   -7.0    0.0 |-123.0 
Матриця А  після 1 кроку
  -6.0   -6.0   15.0    7.0 | 159.0 
   0.0    7.0   -2.5 -0.167 |  34.0 
   0.0   -8.0    1.0 -2.667 | -55.0 
   0.0   -5.0    5.5  5.833 |  10.0 
Матриця А  після 2 кроку
  -6.0   -6.0   15.0    7.0 | 159.0 
   0.0    7.0   -2.5 -0.167 |  34.0 
   0.0    0.0 -1.857 -2.857 | -17.0 
   0.0    0.0  3.714  5.714 |  33.0 
Матриця А  після 3 кроку
  -6.0   -6.0   15.0    7.0 | 159.0 
   0.0    7.0   -2.5 -0.167 |  34.0 
   0.0    0.0 -1.857 -2.857 | -17.0 
   0.0    0.0    0.0    0.0 |  -0.0 
Спроба ділення на нуль у 4 строці
cond=1.2430877232019843e+17


Перевіримо визначник матриці А

In [40]:
print("detA = {}".format(DeterminantGaussian(A3)))

1 та 4 рядки поміняні місцями
  -6.0   -6.0   15.0    7.0 |   0.0 
  -1.0    6.0    0.0    1.0 |   0.0 
   2.0   -6.0   -4.0   -5.0 |   0.0 
   5.0    0.0   -7.0    0.0 |   0.0 
Матриця А  після 1 кроку
  -6.0   -6.0   15.0    7.0 |   0.0 
   0.0    7.0   -2.5 -0.167 |   0.0 
   0.0   -8.0    1.0 -2.667 |   0.0 
   0.0   -5.0    5.5  5.833 |   0.0 
Матриця А  після 2 кроку
  -6.0   -6.0   15.0    7.0 |   0.0 
   0.0    7.0   -2.5 -0.167 |   0.0 
   0.0    0.0 -1.857 -2.857 |   0.0 
   0.0    0.0  3.714  5.714 |   0.0 
Матриця А  після 3 кроку
  -6.0   -6.0   15.0    7.0 |   0.0 
   0.0    7.0   -2.5 -0.167 |   0.0 
   0.0    0.0 -1.857 -2.857 |   0.0 
   0.0    0.0    0.0    0.0 |   0.0 
detA = 0


Отже,бачимо, що визначник матриці = 0, а це означає, що вектори є лінійно залежними і система не має розв'язку, а матриця А не має оберненої


In [41]:
L3,U3 = LUfactorization(A3)
print_matrix(L3,"Ліва нижня трикутна матриця")
print_matrix(U3,"Права верхня трикутна матриця")

Ліва нижня трикутна матриця
   1.0    0.0    0.0    0.0 
  -0.2    1.0    0.0    0.0 
   0.4   -1.0    1.0    0.0 
  -1.2   -1.0   -2.0    1.0 
Права верхня трикутна матриця
   5.0    0.0   -7.0    0.0 
   0.0    6.0   -1.4    1.0 
   0.0    0.0   -2.6   -4.0 
   0.0    0.0    0.0    0.0 


Перевіримо, чи дійсно знайдені матриці є LU розкладом початкової матриці А

In [42]:
print_matrix(L3.dot(U3),'Добуток матриць L та U')
print_matrix(A3, "Очікувана матриця А")

Добуток матриць L та U
   5.0    0.0   -7.0    0.0 
  -1.0    6.0    0.0    1.0 
   2.0   -6.0   -4.0   -5.0 
  -6.0   -6.0   15.0    7.0 
Очікувана матриця А
   5.0    0.0   -7.0    0.0 
  -1.0    6.0    0.0    1.0 
   2.0   -6.0   -4.0   -5.0 
  -6.0   -6.0   15.0    7.0 


Знайдемо обернену матрицю

In [43]:
# print_matrix(linalg.inv(A3),"Обернена матриця, знайдена пакетом ")

In [44]:
# print_matrix(InvertedMatrixLU(A3),"")

Введемо невеликі збурення до кожної з систем

In [45]:
b1_ = b1*0.99

In [46]:
res_ = Gaussian(A1,b1_)
print_vector(res_,"Отриманий розв'язок збуреної СЛАР")
print_vector(res,"Отриманий розв'язок початкової СЛАР")
print("Число обумовленості: {}".format(cond1))

Початкова розширена матриця
   2.0    0.0    3.0    1.0 |  20.0 
  -4.0    3.0   -4.0   -2.0 | -34.0 
   4.0    7.0    9.0    1.0 |  48.0 
   5.0    7.0    0.0    8.0 |  96.0 
1 та 4 рядки поміняні місцями
   5.0    7.0    0.0    8.0 |  96.0 
  -4.0    3.0   -4.0   -2.0 | -34.0 
   4.0    7.0    9.0    1.0 |  48.0 
   2.0    0.0    3.0    1.0 |  20.0 
Матриця А  після 1 кроку
   5.0    7.0    0.0    8.0 |  96.0 
   0.0    8.6   -4.0    4.4 |  43.0 
   0.0    1.4    9.0   -5.4 | -29.0 
   0.0   -2.8    3.0   -2.2 | -19.0 
Матриця А  після 2 кроку
   5.0    7.0    0.0    8.0 |  96.0 
   0.0    8.6   -4.0    4.4 |  43.0 
   0.0    0.0  9.651 -6.116 | -36.0 
   0.0    0.0  1.698 -0.767 |  -5.0 
Матриця А  після 3 кроку
   5.0    7.0    0.0    8.0 |  96.0 
   0.0    8.6   -4.0    4.4 |  43.0 
   0.0    0.0  9.651 -6.116 | -36.0 
   0.0    0.0    0.0  0.308 |   2.0 
Отриманий розв'язок збуреної СЛАР
  6.93
  1.98
   0.0
  5.94
Отриманий розв'язок початкової СЛАР
   7.0
   2.0
  -0.0
   6.0
Ч

In [47]:
b2_ = b2*0.99
res2_ = Gaussian(A2,b2_)
print_vector(res2_,"Отриманий розв'язок збуреної СЛАР")
print_vector(res2,"Отриманий розв'язок початкової СЛАР")
print("Число обумовленості : {}".format(cond2))

Початкова розширена матриця
  -5.0    0.0    7.0    0.0 | 108.0 
   4.0  -24.0    0.0    1.0 | 166.0 
   3.0   12.0   -7.0  -23.0 |-191.0 
  -2.0   42.0   37.0  -21.0 | -94.0 
Матриця А  після 1 кроку
  -5.0    0.0    7.0    0.0 | 108.0 
   0.0  -24.0    5.6    1.0 | 253.0 
   0.0   12.0   -2.8  -23.0 |-126.0 
   0.0   42.0   34.2  -21.0 |-137.0 
2 та 4 рядки поміняні місцями
  -5.0    0.0    7.0    0.0 | 108.0 
   0.0   42.0   34.2  -21.0 |-137.0 
   0.0   12.0   -2.8  -23.0 |-126.0 
   0.0  -24.0    5.6    1.0 | 253.0 
Матриця А  після 2 кроку
  -5.0    0.0    7.0    0.0 | 108.0 
   0.0   42.0   34.2  -21.0 |-137.0 
   0.0    0.0 -12.571  -17.0 | -87.0 
   0.0    0.0 25.143  -11.0 | 174.0 
Матриця А  після 3 кроку
  -5.0    0.0    7.0    0.0 | 108.0 
   0.0   42.0   34.2  -21.0 |-137.0 
   0.0    0.0 -12.571  -17.0 | -87.0 
   0.0    0.0    0.0  -45.0 |   0.0 
Отриманий розв'язок збуреної СЛАР
-11.88
 -8.91
  6.93
  -0.0
Отриманий розв'язок початкової СЛАР
 -12.0
  -9.0
   7.0
   0.0

In [55]:
A1 = np.matrix([ [7.8,  5.3,4.8],
               [3.3,1.1,1.8],
               [4.5,3.3,2.8]])

In [60]:
A1diag = np.matrix([ [7.8,  5.3,4.8],
               [4.5,3.3,2.8],
               [3.3,1.1,1.8]])
b1diag = np.array([1.8,3.4,2.3])

In [61]:
Gaussian(A1diag,b1diag)

Початкова розширена матриця
   7.8    5.3    4.8 |   2.0 
   4.5    3.3    2.8 |   3.0 
   3.3    1.1    1.8 |   2.0 
Матриця А  після 1 кроку
   7.8    5.3    4.8 |   2.0 
   0.0  0.242  0.031 |   2.0 
   0.0 -1.142 -0.231 |   2.0 
Матриця А  після 2 кроку
   7.8    5.3    4.8 |   2.0 
   0.0  0.242  0.031 |   2.0 
   0.0    0.0 -0.086 |  13.0 


array([  71.82716049,   28.51851852, -147.83333333])